# stackview
Interactive image stack viewing in jupyter notebooks based on 
[numpy_image_widget](https://github.com/Who8MyLunch/Numpy_Image_Widget) and 
[ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).

## Installation

In [ ]:
!pip install stackview

## Usage
You can use `stackview` from within jupyter notebooks as shown below.

Starting point is a 3D image dataset provided as numpy array. 

In [1]:
import stackview
from skimage.io import imread
from skimage.filters import gaussian

In [2]:
image = imread('https://github.com/haesleinhuepf/stackview/blob/main/docs/data/Haase_MRT_tfl3d1.tif?raw=true', plugin='tifffile')

You can then view it slice-by-slice:

In [3]:
stackview.slice(image, continuous_update=True)

Orthogonal views are also available:

In [4]:
stackview.orthogonal(image, continuous_update=True)

Furthermore, to visualize an original image in combination with a processed version, a curtain view may be helpful:

In [5]:
modified_image = gaussian(image, sigma=3)

In [6]:
stackview.curtain(image, modified_image * 65537, continuous_update=True)